# Using NGINX for demos on microk8s - ie - guestbook ...

1. 

In [1]:
import os
my_out = os.system('ls')

In [2]:
my_out

0

In [3]:
import subprocess

cmd = "date"

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current date is:', returned_output.decode("utf-8"))

Current date is: Mon Mar  9 01:27:15 UTC 2020



In [4]:
cmd = "ls"

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current LS STUFF is:\n', returned_output.decode("utf-8"))

Current LS STUFF is:
 00-nginx-reverse.ipynb
99-using-traefik-for-class.ipynb
docker-compose.yml
nginx
traefik.toml



In [5]:
cmd = "/snap/bin/microk8s.status"

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current LS STUFF is:\n', returned_output.decode("utf-8"))

Current LS STUFF is:
 microk8s is running
addons:
cilium: disabled
dashboard: enabled
dns: enabled
fluentd: disabled
gpu: disabled
helm: disabled
ingress: disabled
istio: disabled
jaeger: disabled
juju: disabled
knative: disabled
kubeflow: disabled
linkerd: disabled
metallb: disabled
metrics-server: disabled
prometheus: disabled
rbac: disabled
registry: disabled
storage: disabled



In [6]:
cmd = ["/snap/bin/microk8s.kubectl", "--help"]

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current LS STUFF is:\n', returned_output.decode("utf-8"))

Current LS STUFF is:
 kubectl controls the Kubernetes cluster manager.

 Find more information at: https://kubernetes.io/docs/reference/kubectl/overview/

Basic Commands (Beginner):
  create         Create a resource from a file or from stdin.
  expose         Take a replication controller, service, deployment or pod and expose it as a new Kubernetes Service
  run            Run a particular image on the cluster
  set            Set specific features on objects

Basic Commands (Intermediate):
  explain        Documentation of resources
  get            Display one or many resources
  edit           Edit a resource on the server
  delete         Delete resources by filenames, stdin, resources and names, or by resources and label selector

Deploy Commands:
  rollout        Manage the rollout of a resource
  scale          Set a new size for a Deployment, ReplicaSet or Replication Controller
  autoscale      Auto-scale a Deployment, ReplicaSet, or ReplicationController

Cluster Management

In [7]:
cmd = ["/snap/bin/microk8s.kubectl", "get", 'svc', '-l tier=frontend']

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current LS STUFF is:\n', returned_output.decode("utf-8"))

Current LS STUFF is:
 NAME       TYPE       CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
frontend   NodePort   10.152.183.182   <none>        80:30537/TCP   19h



In [8]:
just_fe = returned_output.decode("utf-8") 
just_fe = just_fe.split('\n')[1]
just_fe

'frontend   NodePort   10.152.183.182   <none>        80:30537/TCP   19h'

In [9]:
import re
ports = re.split('\s+',just_fe)

In [10]:
ports

['frontend', 'NodePort', '10.152.183.182', '<none>', '80:30537/TCP', '19h']

In [11]:
nodeport = ports[4]
nodeport

'80:30537/TCP'

In [12]:
nodeport = nodeport.split(':')[1]

In [13]:
nodeport

'30537/TCP'

In [14]:
nodeport = nodeport.split('/')[0]

In [15]:
nodeport

'30537'

In [16]:
cmd = ["ip", "a", 'show', 'ens5']

# returns output as byte string
returned_output = subprocess.check_output(cmd)

# using decode() function to convert byte string to string
print('Current LS STUFF is:\n', returned_output.decode("utf-8"))

Current LS STUFF is:
 2: ens5: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9001 qdisc mq state UP group default qlen 1000
    link/ether 02:4a:23:b8:cb:64 brd ff:ff:ff:ff:ff:ff
    inet 10.12.68.23/23 brd 10.12.69.255 scope global dynamic ens5
       valid_lft 3003sec preferred_lft 3003sec
    inet6 fe80::4a:23ff:feb8:cb64/64 scope link 
       valid_lft forever preferred_lft forever



In [17]:
just_fe = returned_output.decode("utf-8") 
just_fe = just_fe.split('\n')[2]
just_fe

'    inet 10.12.68.23/23 brd 10.12.69.255 scope global dynamic ens5'

In [18]:
ips = re.split('\s+',just_fe)

In [19]:
ips

['',
 'inet',
 '10.12.68.23/23',
 'brd',
 '10.12.69.255',
 'scope',
 'global',
 'dynamic',
 'ens5']

In [20]:
my_ip = ips[2]
my_ip

'10.12.68.23/23'

In [21]:
my_ip = my_ip.split('/')[0]

In [22]:
my_ip

'10.12.68.23'

In [23]:
print(my_ip)

10.12.68.23


In [24]:
def modify_kub_conf(version):
    """ reads docker file and will increment version"""
    # Open the file with read only permit
    f = open('./nginx/nginx/conf.d/kub.conf', "r")
    # use readlines to read all lines in the file
    # The variable "lines" is a list containing all lines in the file
    lines = f.readlines()
    # close the file after reading the lines.
    for line in lines:
        print(line)
        if version in line:
            print(line)
            #increment_line(line)
    f.close()
    print(version)

In [25]:
version='1.1'
modify_kub_conf(version)

upstream k8s {

  server        10.12.68.23:30537;

}



server {

  listen        80;

  server_name   k8s.tony.com;



  location / {

    proxy_pass  http://10.12.68.23:30537;

  }

}

1.1


In [26]:
!ls nginx/nginx/conf.d

kub.conf  sites-available  sites-enabled


In [27]:
kub_con = """
server {
  listen        80;
  server_name   k8s.tony.com;

  location / {
    #proxy_pass  http://10.12.68.23:30537;
  }
}

"""

In [29]:
print (kub_con)


server {
  listen        80;
  server_name   k8s.tony.com;

  location / {
    #proxy_pass  http://10.12.68.23:30537;
  }
}




In [35]:
kub_con = """
server {
  listen        80;
  server_name   k8s.tony.com;

  location / {
    #proxy_pass  http://10.12.68.23:30537;
    %s
  }
}

"""

In [36]:
print(kub_con)


server {
  listen        80;
  server_name   k8s.tony.com;

  location / {
    #proxy_pass  http://10.12.68.23:30537;
    %s
  }
}




In [37]:
proxy_str = "proxy_pass http://%s:%s" % (my_ip, nodeport)
proxy_str

'proxy_pass http://10.12.68.23:30537'

In [38]:
kub_con = kub_con % (proxy_str)

In [39]:
kub_con

'\nserver {\n  listen        80;\n  server_name   k8s.tony.com;\n\n  location / {\n    #proxy_pass  http://10.12.68.23:30537;\n    proxy_pass http://10.12.68.23:30537\n  }\n}\n\n'

In [40]:
! ls nginx/nginx/conf.d/kub.conf

nginx/nginx/conf.d/kub.conf


In [41]:
filename = 'nginx/nginx/conf.d/kub.conf'
f = open(filename, "w")
f.write(kub_con)
f.close()

In [42]:
version='1.1'
modify_kub_conf(version)



server {

  listen        80;

  server_name   k8s.tony.com;



  location / {

    #proxy_pass  http://10.12.68.23:30537;

    proxy_pass http://10.12.68.23:30537

  }

}



1.1


In [44]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [45]:
! $k get svc

NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
frontend           NodePort    10.152.183.182   <none>        80:30537/TCP   19h
kubernetes         ClusterIP   10.152.183.1     <none>        443/TCP        2d1h
microbot-service   NodePort    10.152.183.195   <none>        80:31914/TCP   46h
redis-master       ClusterIP   10.152.183.198   <none>        6379/TCP       20h
redis-slave        ClusterIP   10.152.183.78    <none>        6379/TCP       20h
